In [1]:
!unzip weights-qlora.zip

Archive:  weights-qlora.zip
   creating: weights-qlora/
  inflating: weights-qlora/preprocessor_config.json  
  inflating: weights-qlora/vocab.txt  
  inflating: weights-qlora/adapter_model.safetensors  
  inflating: weights-qlora/tokenizer_config.json  
  inflating: weights-qlora/adapter_config.json  
  inflating: weights-qlora/tokenizer.json  
  inflating: weights-qlora/README.md  
  inflating: weights-qlora/special_tokens_map.json  


In [2]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [3]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

siddhesh1712_vrdataset_path = kagglehub.dataset_download('siddhesh1712/vrdataset')
siddhesh1712_train_test_val_path = kagglehub.dataset_download('siddhesh1712/train-test-val')
print('Data source import complete.')


100%|██████████| 2.73G/2.73G [02:15<00:00, 21.6MB/s]

Extracting files...


100%|██████████| 2.09M/2.09M [00:01<00:00, 2.07MB/s]

Extracting files...


Data source import complete.


In [5]:
import pandas as pd

data = pd.read_csv(siddhesh1712_train_test_val_path+'/test.csv')
data.head(5)

,image_id,image_path,question,response
0,61BorBlKJmL,images/small/c3/c34ecbb9.jpg,What is the color of the phone case?,Orange
1,81Ba3iri9eL,images/small/56/565b61e4.jpg,"What is the heel's measurement, in inches?",One
2,416CFhHPU6L,images/small/8b/8b497642.jpg,Is the cup designed for cold beverages?,No
3,41VkOGAbdAL,images/small/66/666e46a0.jpg,Where was it made?,China
4,61Ad3nx6IAL,images/small/c0/c089d4ac.jpg,What phone models is the case for?,M21


In [6]:
from transformers import BlipProcessor, BlipForQuestionAnswering
from peft import PeftModel
from PIL import Image
import torch

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Base model ID (must match the base model used during QLoRA training)
base_model_id = "Salesforce/blip-vqa-base"  # or the actual base you trained on

# Load processor (same as base model)
processor = BlipProcessor.from_pretrained(base_model_id)

# Load base model (quantized version is not typical with BLIP, but use FP16 if needed)
model = BlipForQuestionAnswering.from_pretrained(
    base_model_id,
    torch_dtype=torch.float16,
).to(device)

# Load QLoRA adapter weights using PEFT
qlora_adapter_path = "weights-qlora"
model = PeftModel.from_pretrained(model, qlora_adapter_path)
model.eval()


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

PeftModel(
  (base_model): LoraModel(
    (model): BlipForQuestionAnswering(
      (vision_model): BlipVisionModel(
        (embeddings): BlipVisionEmbeddings(
          (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
        )
        (encoder): BlipEncoder(
          (layers): ModuleList(
            (0-11): 12 x BlipEncoderLayer(
              (self_attn): BlipAttention(
                (dropout): Dropout(p=0.0, inplace=False)
                (qkv): Linear(in_features=768, out_features=2304, bias=True)
                (projection): Linear(in_features=768, out_features=768, bias=True)
              )
              (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
              (mlp): BlipMLP(
                (activation_fn): GELUActivation()
                (fc1): Linear(in_features=768, out_features=3072, bias=True)
                (fc2): Linear(in_features=3072, out_features=768, bias=True)
              )
              (layer_norm2): Lay

In [7]:
def help(image_path,question):
  try:
      image = Image.open(image_path).convert("RGB")
  except:
      return "error"
  # prompt = f"Question:{question} Answer:"
  inputs = processor(images=image, text=question, return_tensors="pt").to(device, torch.float16)
  generated_ids = model.generate(**inputs,max_new_tokens=10,use_cache=True)
  generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
  return generated_text

In [8]:
import re
predictions = []
c=0
for idx, row in data.iterrows():
    image_path = str(row['image_path'])
    question = str(row['question'])
    raw_answer = help(siddhesh1712_vrdataset_path+"/"+image_path, question)
    predictions.append(raw_answer)

In [9]:
print(predictions)

['orange', 'thirty', 'yes', 'india', 'oneplus', 'blue', 'cotton', 'black', 'lentil', 'lace', 'modern', 'lace', 'cheese', 'vivo', 'yes', 'no', 'almonds', 'phone', 'heart', 'three', 'no', 'yoga', 'wholefoods', 'yes', 'samsung', 'two', 'yes', 'round', 'wholefoods', 'no', 'stylish', 'three', 'white', 'yes', 'yes', 'protection', 'hard', 'no', 'queen', 'yes', 'polyester', 'zipper', 'year', 'yes', 'metal', 'no', 'rectangular', 'diamonds', 'yes', 'wood', 'stylish', 'case', 'suede', 'europe', 'wool', 'samsung', 'linen', 'plastic', 'samsung', 'hard', 'diamond', 'lenovo', 'round', 'converse', 'yes', 'gold', 'basic', 'sol', 'two', 'stainless', 'year', 'thirty', 'wood', 'mattress', 'air', 'hard', 'stylish', 'three', 'grade', 'lace', 'copper', 'gionee', 'canada', '100', 'ceramic', 'round', 'embossed', 'one on top', 'three', 'amazon', 'yes', 'ceiling fan', 'yes', 'garlic', 'orange', 'fifteen', 'two', 'china', 'twenty', 'nokia', 'pyro', 'semicircle', 'tan', 'samsung', 'iphone', 'no', 'pomegranate', 's

In [10]:
# import re
# from PIL import Image

# batch_size = 256
# batched_preds = []
# questions=[]
# images=[]
# for idx,row in data.iterrows():
#     image_path = row['image_path']
#     questions.append(f"Question: {str(row['question'])} Answer:")
#     image = Image.open("/kaggle/input/vrdataset/" + image_path).convert("RGB")
#     images.append(image)

In [11]:
# # model.generation_config.cache_implementation = "offloaded"
# c=0
# for i in range(0, len(data), batch_size):
#     img = images[i:i+batch_size]
#     qst = questions[i:i+batch_size]

#     # Tokenize with appropriate padding/truncation
#     inputs = processor(images=img, text=qst, return_tensors="pt", padding=True).to(device, torch.float16)

#     generated_ids = model.generate(**inputs,max_new_tokens=20,cache_implementation="offloaded")

#     outputs = processor.batch_decode(generated_ids, skip_special_tokens=True)
#     print(outputs)
#     batched_preds.extend([ans.strip() for ans in outputs])
# #     batched_preds.extend([
# #     ans.split("Answer:")[1].strip() if "Answer:" in ans and ans.split("Answer:")[1].strip() else ""
# #     for ans in outputs
# # ])
#     c+=1
#     if c==1:
#         break

# # - 31.5 minute

In [12]:
# batched_preds

In [13]:
empty_count = predictions.count('')
print("Number of empty strings:", empty_count)

Number of empty strings: 3


In [14]:
!pip install bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [15]:
from bert_score import score

references = data['response'].astype(str).tolist()

P, R, F1 = score(predictions, references, lang="en")

# Average F1
print(f"Mean BERTScore F1: {F1.mean().item():.4f}")
print(f"Mean BERTScore Precision: {P.mean().item():.4f}")
print(f"Mean BERTScore Recall   : {R.mean().item():.4f}")

# ------- Base-Line ------
# Mean BERTScore F1: 0.8933
# Mean BERTScore Precision: 0.8859
# Mean BERTScore Recall   : 0.9030


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Mean BERTScore F1: 0.9328
Mean BERTScore Precision: 0.9412
Mean BERTScore Recall   : 0.9266


In [16]:
def normalize(text):
    return str(text).strip().lower()

norms_pred = predictions.copy()

# Apply normalization
for i in norms_pred:
  i = normalize(i)
normalized_truths = data['response'].apply(normalize)

# Compare for exact match
k = norms_pred == normalized_truths

# Compute accuracy
exact_match_accuracy = k.mean()
print(f"Exact Match Accuracy: {exact_match_accuracy:.5f}")

# Exact Match Accuracy: 0.19072 - BaseLine -----

Exact Match Accuracy: 0.40544


In [17]:
!pip install sentence-transformers

In [18]:
from sentence_transformers import SentenceTransformer, util

# Load a lightweight and suitable model for short text
model = SentenceTransformer('all-MiniLM-L6-v2')

# Convert to embeddings
pred_embeds = model.encode(predictions, convert_to_tensor=True)
ref_embeds = model.encode(references, convert_to_tensor=True)

# Compute cosine similarities for each pair
cosine_scores = util.cos_sim(pred_embeds, ref_embeds)

# Average score (diagonal contains the matching prediction-reference pairs)
average_cosine_score = cosine_scores.diag().mean().item()
print(f"Average Cosine Similarity Score: {average_cosine_score:.4f}")

# Average Cosine Similarity Score: 0.515 -- BaseLine

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Average Cosine Similarity Score: 0.6654


In [19]:
# Cosine similarity as pass/fail
correct = (cosine_scores.diag() >= 0.75).sum().item()
accuracy = correct / len(predictions)
accuracy

# BaseLine : 0.24336 - Best Consine Match

0.4422